# Computerphysik Programmiertutorial 5b
Prof. Dr. Matteo Rizzi und Dr. Markus Schmitt - Institut für Theoretische Physik, Universität zu Köln
&nbsp;

**Github**: [https://github.com/markusschmitt/compphys2021](https://github.com/markusschmitt/compphys2021)

**Inhalt dieses Notebooks**: Timing (`BenchmarkTools`), Komplexität


## Timing

Bei wissenschaftlichen Programmen kommt es oft auf die Effizienz an. Um die zu optimieren ist es wichtig die Resourcen bestimmen zu können, die zum Ausführen des Programms benötigt werden. Die wichtigen Resourcen sind Speicherplatz und Laufzeit. Wir werden uns hier auf Zeitmessungen beschränken.

In [ ]:
function f(x)
    for j in eachindex(x)
        x[j] += 3.7
    end

    return nothing
end

Das Makro `@time` wird einem Funktionsaufruf vorangestellt um eine Ausgabe der verwendeten Resourcen zu erhalten:

Achtung: Der erste Funktionsaufruf dauert länger, weil die Funktion dann [just-in-time-kompiliert](https://de.wikipedia.org/wiki/Just-in-time-Kompilierung) (jit-kompiliert) wird.

`@elapsed` gibt nur die Laufzeit (in Sekunden) zurück.

Mit dem Paket `BenchmarkTools` können genauere Messungen vorgenommen werden

## Komplexität

In [ ]:
using Plots
using LinearAlgebra

Zur Anschauung definieren wir eine eigene Funktion für Matrixmultiplikation:

In [ ]:
function my_matmul(A,B)
    C = Array{Float64}(undef,size(A)[1], size(B)[2])

    for j in 1:size(C)[2]
        for i in 1:size(C)[1]
            C[i,j] = 0.0
            for k in 1:size(A)[2]
                C[i,j] += A[i,k] * B[k,j]
            end
        end
    end

    return C
end

Benchmark unserer Funktion:

Benchmark der vorimplementierten Matrixmultiplikation:

Schauen wir uns das etwas systematischer für verschiedene Matrixgrößen an:

In [ ]:
BLAS.set_num_threads(1);

ns=2 .^ (collect(5:10))

times1=Float64[]
times2=Float64[]

for n in ns
    # zwei Matrizen erstellen
    A=rand(n,n)
    B=rand(n,n)
    
    # Zeit für Multiplikation messen
    t1 = @elapsed A*B
    t2 = @elapsed my_matmul(A,B)
    
    # Ausgabe
    println("n=$n:")
    println("  Zeit für A*B: $(t1)s")
    println("  Zeit für my_matmul(A,B): $(t2)s")
    
    # Messergebnis an Arrays anhängen
    push!(times1, t1)
    push!(times2, t2)
end

In [ ]:
plot(ns,times1, marker=:o, label="*")
plot!(ns,times2, marker=:o, label="my_matmul")
xlabel!("Matrizengröße n")
ylabel!("Rechenzeit [s]")

Die **Komplexität** eines Algorithmus gibt an wie die benötigten *Rechenresourcen* (Rechenzeit oder Speicherplatz) wachsen, wenn man das zu lösende Problem vergrößert. In unserem Beispiel ist die entspricht die Größe des Problems der Größe unserer $n\times n$-Matrizen.

Um eine Gesetzmäßigkeit zu finden, schauen wir noch einmal genauer hin, indem wir doppelt-logarithmisch plotten:

In [ ]:
plot(ns,times1, marker=:o, xaxis=:log, yaxis=:log, label="*")
plot!(ns,times2, marker=:o, xaxis=:log, yaxis=:log, label="my_matmul")
xlabel!("Matrizengröße n")
ylabel!("Rechenzeit [s]")

Die Komplexität von Algorithmen wird in $O$-Notation angegeben. Die Multiplikation von zwei $n\times n$ Matrizen wie wir sie hier implementiert haben ist z.B. $O(n^3)$. Das bedeutet, dass die Rechenzeit bei großen Matrizen kubisch wächst. Der Vorfaktor dieses Wachstumsgesetzes kann jedoch von den Details der Implementierung abhängen.

Eine Zusammenfassung der Komplexität bekannter Algorithmen steht auf [Wikipedia](https://en.wikipedia.org/wiki/Computational_complexity_of_mathematical_operations#Matrix_algebra).

In [ ]:
ns=2 .^ (collect(5:10))
times1=Float64[]
times2=Float64[]

for n in ns
    # Matrix und Vektor erstellen
    A=rand(n,n)
    B=rand(n,1) # Diesmal Matrix-Vektor Produkt
    
    # Zeit für Multiplikation messen
    t1 = @elapsed A*B
    t2 = @elapsed my_matmul(A,B)
    
    # Ausgabe
    println("n=$n:")
    println("  Zeit für A*B: $t1")
    println("  Zeit für my_matmul(A,B): $t2")
    
    # Messergebnis an Arrays anhängen
    push!(times1, t1)
    push!(times2, t2)
end

In [ ]:
using LaTeXStrings

plot(ns,times1, marker=:o, xaxis=:log, yaxis=:log, label="*")
plot!(ns,times2, marker=:o, xaxis=:log, yaxis=:log, label="my_matmul")
plot!(ns, 5e-10*ns.^2, linestyle=:dash, xaxis=:log, yaxis=:log, label=L"n^2")
xlabel!("Matrizengröße n")
ylabel!("Rechenzeit")